In [1]:
import dateutil.relativedelta as relativedelta
import dateutil.rrule as rrule
import datetime

import numpy as np
import pandas as pd
import pandas_profiling

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [2]:
rs_card_data = pd.read_csv('../../data/bigcontest/02_카드매출데이터(신한카드)/CARD_SPENDING_RESIDENT/CARD_SPENDING_RESIDENT.txt', sep='\t')
fr_card_data = pd.read_csv('../../data/bigcontest/02_카드매출데이터(신한카드)/CARD_SPENDING_FOREIGNER/CARD_SPENDING_FOREIGNER.txt', sep='\t', engine='python')

In [71]:
covid_confirmed = pd.read_csv('../../data/covid19/covid19_confirmed.csv').drop('Unnamed: 0',axis=1)

In [72]:
covid_confirmed['month'] = [int(str(cnt).split('.')[0]) for cnt in covid_confirmed.date]
covid_confirmed['day'] = [int(str(cnt).split('.')[1]) for cnt in covid_confirmed.date]

In [73]:
covid_confirmed = covid_confirmed.melt(id_vars=['date','month','day'],  var_name='city', value_name='covid19')

In [74]:
covid_confirmed.head()

,date,month,day,city,covid19
0,2.18,2,18,seoul,0
1,2.19,2,19,seoul,1
2,2.20,2,2,seoul,7
3,2.21,2,21,seoul,5
4,2.22,2,22,seoul,3


# 전처리
## mct_cat_cd 한글로 변환

In [7]:
mct_dict = { 10:'숙박', 20:'레저용품', 21:'레저업소', 22:'문화취미',
 30:'가구', 31:'전기', 32:'주방용구', 33:'연료판매',
 34:'광학제품', 35:'가전', 40:'유통업', 42:'의복',
 43:'직물', 44:'신변잡화', 50:'서적문구', 52:'사무통신',
 60:'자동차판매', 62:'자동차정비', 70:'의료기관', 71:'보건위생',
 80:'요식업소', 81:'음료식품', 92:'수리서비스' }

In [8]:
rs_card_data.MCT_CAT_CD = [mct_dict[cnt] for cnt in rs_card_data.MCT_CAT_CD]

## x축

In [9]:
# x tics
year =2020
before =datetime.datetime(year,1,1)
after =datetime.datetime(year,12,31)
rr = rrule.rrule(rrule.WEEKLY,byweekday=relativedelta.SU,dtstart=before)

test = pd.DataFrame(rr.between(before,after,inc=True))  + datetime.timedelta(days=-13) # 월요일 기준

test2 = list(map(lambda x : str(pd.to_datetime(x[0]).month)+'/'+ str(pd.to_datetime(x[0]).day)
         , list(test.values)))

## rs_card_data 시간 분할

In [10]:
rs_card_data.STD_DD = pd.to_datetime(rs_card_data.STD_DD,format='%Y%m%d')

rs_card_data.SEX_CD = rs_card_data.SEX_CD.astype('category')

In [11]:
rs_card_data['year'] = rs_card_data.STD_DD.dt.year
rs_card_data['month'] = rs_card_data.STD_DD.dt.month
rs_card_data['day'] = rs_card_data.STD_DD.dt.day
rs_card_data['week'] = rs_card_data.STD_DD.dt.weekofyear 

## 서울 대구 분리

In [12]:
rs_card_data

,STD_DD,GU_CD,DONG_CD,MCT_CAT_CD,SEX_CD,AGE_CD,USE_CNT,USE_AMT,year,month,day,week
0,2019-02-01,110,517,숙박,F,25,11,595,2019,2,1,5
1,2019-02-01,110,517,숙박,F,50,5,243,2019,2,1,5
2,2019-02-01,110,517,숙박,F,55,5,27,2019,2,1,5
3,2019-02-01,110,517,숙박,M,20,11,243,2019,2,1,5
4,2019-02-01,110,517,숙박,M,25,11,433,2019,2,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...
2537748,2020-05-31,350,720,음료식품,M,45,19,127,2020,5,31,22
2537749,2020-05-31,350,720,음료식품,M,50,33,211,2020,5,31,22
2537750,2020-05-31,350,720,음료식품,M,55,9,61,2020,5,31,22
2537751,2020-05-31,350,720,음료식품,M,60,19,194,2020,5,31,22


In [13]:
gu_dict = { 140:'seoul', 350:'seoul', 260:'daegu', 110:'daegu' }

In [14]:
rs_card_data.GU_CD = [gu_dict[cnt] for cnt in rs_card_data.GU_CD]

In [15]:
rs_card_data.columns = ['STD_DD', 'city', 'DONG_CD', 'MCT_CAT_CD', 'SEX_CD', 'AGE_CD',
       'USE_CNT', 'USE_AMT', 'year', 'month', 'day', 'week']

In [16]:
rs_card_data.head()

,STD_DD,city,DONG_CD,MCT_CAT_CD,SEX_CD,AGE_CD,USE_CNT,USE_AMT,year,month,day,week
0,2019-02-01,daegu,517,숙박,F,25,11,595,2019,2,1,5
1,2019-02-01,daegu,517,숙박,F,50,5,243,2019,2,1,5
2,2019-02-01,daegu,517,숙박,F,55,5,27,2019,2,1,5
3,2019-02-01,daegu,517,숙박,M,20,11,243,2019,2,1,5
4,2019-02-01,daegu,517,숙박,M,25,11,433,2019,2,1,5


# 시작

In [82]:
table = rs_card_data.pivot_table(values=['USE_CNT', 'USE_AMT'],
                                 index=['month', 'day', 'week',
                                        'MCT_CAT_CD', 'city', 'year'],
                                 aggfunc='sum').reset_index()

In [83]:
covid_table = pd.merge(table, covid_confirmed,on=['month','day', 'city'], how='left').fillna('0')

In [141]:
covid_table['covid19'] = covid_table['covid19'].astype('int')

In [142]:
covid_table2020 = covid_table[covid_table['year']==2020].drop('year',axis=1)

In [143]:
seoul_covid_table2020 = covid_table2020[covid_table2020['city']=='seoul'].drop('city',axis=1)
daegu_covid_table2020 = covid_table2020[covid_table2020['city']=='daegu'].drop('city',axis=1)

In [146]:
seoul_covid_table2020.pivot_table(values=['covid19','USE_AMT'],
                         index=['week', 'MCT_CAT_CD'],
                         aggfunc='sum').reset_index()

,week,MCT_CAT_CD,USE_AMT,covid19
0,5,가구,123278,7
1,5,가전,447355,7
2,5,광학제품,60630,7
3,5,레저업소,596608,7
4,5,레저용품,1385754,7
...,...,...,...,...
409,22,자동차정비,1366926,84
410,22,자동차판매,177114,84
411,22,전기,36289,84
412,22,주방용구,698136,84


In [187]:
def elastic(data, week):
    data = data.set_index('MCT_CAT_CD')
    last_week = data[data['week']==week-1][['week','USE_AMT','covid19']]
    this_week = data[data['week']==week][['week','USE_AMT','covid19']]
    return ( (this_week.USE_AMT - last_week.USE_AMT)/ last_week.USE_AMT ) / ( (this_week.covid19 - last_week.covid19)/ last_week.covid19 )

In [198]:
data = seoul_covid_table2020
week = 11



In [199]:
data = data.set_index('MCT_CAT_CD')
last_week = data[data['week']==week-1][['week','USE_AMT','covid19']].drop_duplicates()
this_week = data[data['week']==week][['week','USE_AMT','covid19']].drop_duplicates()

In [203]:
this_week

,week,USE_AMT,covid19
MCT_CAT_CD,,,
가구,11,63286,10
가전,11,165605,10
광학제품,11,30488,10
레저업소,11,217882,10
레저용품,11,332676,10
...,...,...,...
의복,11,145505,9
자동차정비,11,28033,9
자동차판매,11,6901,9


In [204]:
last_week

,week,USE_AMT,covid19
MCT_CAT_CD,,,
가구,10,40496,4
가구,10,40496,17
가전,10,262791,4
가전,10,262791,17
광학제품,10,62269,4
...,...,...,...
자동차정비,10,33967,12
자동차판매,10,16571,12
전기,10,4540,12


In [201]:
this_week.USE_AMT - last_week.USE_AMT

MCT_CAT_CD
가구    22790
가구    22790
가구    46266
가구    46266
가구    39420
      ...  
직물    -9473
직물   -24731
직물   -21188
직물   -12138
직물     4618
Name: USE_AMT, Length: 1440, dtype: int64

In [200]:
((this_week.USE_AMT - last_week.USE_AMT)/ last_week.USE_AMT ) / ( (this_week.covid19 - last_week.covid19)/ last_week.covid19 )

MCT_CAT_CD
가구    0.375181
가구    1.594520
가구    1.562182
가구    3.570701
가구    0.159152
        ...   
직물   -0.083131
직물   -0.182304
직물   -0.101835
직물   -0.218015
직물   -4.173520
Length: 12960, dtype: float64

In [197]:
last_week.USE_AMT

MCT_CAT_CD
가구        40496
가구        40496
가전       262791
가전       262791
광학제품      62269
          ...  
자동차정비     33967
자동차판매     16571
전기         4540
주방용구      18754
직물         4426
Name: USE_AMT, Length: 207, dtype: int64

In [188]:
elastic(data=seoul_covid_table2020, week=11)

MCT_CAT_CD
가구    0.375181
가구    1.594520
가구    1.562182
가구    3.570701
가구    0.159152
        ...   
직물   -0.083131
직물   -0.182304
직물   -0.101835
직물   -0.218015
직물   -4.173520
Length: 12960, dtype: float64

MCT_CAT_CD
가구    0.375181
가구    1.594520
가구    1.562182
가구    3.570701
가구    0.159152
        ...   
직물   -0.083131
직물   -0.182304
직물   -0.101835
직물   -0.218015
직물   -4.173520
Length: 12960, dtype: float64

In [176]:
this_week

,week,USE_AMT,covid19
MCT_CAT_CD,,,
가구,7,27442,0
가전,7,132893,0
광학제품,7,29119,0
레저업소,7,344975,0
레저용품,7,451269,0
...,...,...,...
자동차정비,7,31520,0
자동차판매,7,12774,0
전기,7,88,0


In [173]:
this_week

,week,USE_AMT,covid19
MCT_CAT_CD,,,
가구,7,27442,0
가전,7,132893,0
광학제품,7,29119,0
레저업소,7,344975,0
레저용품,7,451269,0
...,...,...,...
자동차정비,7,31520,0
자동차판매,7,12774,0
전기,7,88,0


In [103]:
seoul_covid_table2020

,month,day,week,MCT_CAT_CD,USE_AMT,USE_CNT,date,covid19
3,2,1,5,가구,61331,117,0,0
7,2,1,5,가전,258731,569,0,0
11,2,1,5,광학제품,46706,307,0,0
15,2,1,5,레저업소,332995,21906,0,0
19,2,1,5,레저용품,712649,8886,0,0
...,...,...,...,...,...,...,...,...
11833,5,31,22,자동차정비,72164,6846,5.31,6
11837,5,31,22,자동차판매,70,5,5.31,6
11840,5,31,22,전기,482,10,5.31,6
11844,5,31,22,주방용구,17444,252,5.31,6


In [60]:
table

month day MCT_CAT_CD   city   USE_AMT           USE_CNT       
year                                  2019      2020    2019   2020
0        2   1         가구  daegu    9538.0   60086.0    35.0   70.0
1        2   1         가구  seoul   10706.0   61331.0    59.0  117.0
2        2   1         가전  daegu   63797.0   24895.0   283.0  123.0
3        2   1         가전  seoul  177931.0  258731.0   997.0  569.0
4        2   1       광학제품  daegu   14321.0   49697.0   275.0  465.0
...    ...  ..        ...    ...       ...       ...     ...    ...
5559     5  31         전기  seoul    2736.0     482.0    16.0   10.0
5560     5  31       주방용구  daegu   35512.0   13257.0   148.0  215.0
5561     5  31       주방용구  seoul   44599.0   17444.0   308.0  252.0
5562     5  31         직물  daegu   26565.0   38595.0   276.0  461.0
5563     5  31         직물  seoul  139386.0   34780.0   304.0  260.0

[5564 rows x 8 columns]

In [61]:
table.merge(covid_confirmed,on=['month','day', 'city'])

C:\Users\User\anaconda3\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


ValueError: You are trying to merge on int64 and object columns. If you wish to proceed you should use pd.concat

In [15]:
table.columns

# 20년도 - 19년도 : 즉 증감수치

MultiIndex([('USE_AMT', 2019),
            ('USE_AMT', 2020),
            ('USE_CNT', 2019),
            ('USE_CNT', 2020)],
           names=[None, 'year'])

In [51]:
# 전체 이용금액 차이
total_diff = (table.iloc[:,1] - table.iloc[:,0])

# 건당 이용금액 차이
cnt_diff = (table.iloc[:,1] / table.iloc[:,3] - table.iloc[:,0] / table.iloc[:,2])

In [58]:
# plt.figure(figsize=(30, 10))
# sns.lineplot(data=total_diff.unstack('MCT_CAT_CD'), dashes=False)
# plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
# plt.xticks(ticks=(np.arange(0, len(test2))), labels=test2)
# plt.show()

In [59]:
# plt.figure(figsize=(30, 10))
# sns.lineplot(data=cnt_diff.unstack('MCT_CAT_CD'), dashes=False)
# plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')
# plt.xticks(ticks=(np.arange(0, len(test2))), labels=test2)
# plt.show()

In [60]:
total_diff2 = total_diff.reset_index()
cnt_diff2 = cnt_diff.reset_index()

total_diff2.columns = ['week', 'MCT_CAT_CD', 'diff']
cnt_diff2.columns = ['week', 'MCT_CAT_CD', 'diff']

In [61]:
fig = px.line(total_diff2, x="week", y="diff", color='MCT_CAT_CD')

fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = np.arange(0, len(test2)),
        ticktext = test2
    )
)
fig.show()

In [22]:
fig = px.line(cnt_diff2, x="week", y="diff", color='MCT_CAT_CD')

fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = np.arange(0, len(test2)),
        ticktext = test2
    )
)
fig.show()

In [23]:
rs_card_data[rs_card_data.week== max(rs_card_data.week-1)]
# 즉 5.18의 값은 아래를 의미

,STD_DD,GU_CD,DONG_CD,MCT_CAT_CD,SEX_CD,AGE_CD,USE_CNT,USE_AMT,year,month,day,week
1177416,2019-05-20,110,517,숙박,F,20,6,83,2019,5,20,21
1177417,2019-05-20,110,517,숙박,F,25,6,220,2019,5,20,21
1177418,2019-05-20,110,517,숙박,M,20,6,193,2019,5,20,21
1177419,2019-05-20,110,517,숙박,M,25,11,275,2019,5,20,21
1177420,2019-05-20,110,517,숙박,M,30,17,415,2019,5,20,21
...,...,...,...,...,...,...,...,...,...,...,...,...
2460554,2020-05-24,350,720,음료식품,M,40,33,312,2020,5,24,21
2460555,2020-05-24,350,720,음료식품,M,50,9,106,2020,5,24,21
2460556,2020-05-24,350,720,음료식품,M,55,5,26,2020,5,24,21
2460557,2020-05-24,350,720,음료식품,M,60,14,249,2020,5,24,21


# 서울 대구 분리

In [24]:
data_seoul = rs_card_data[rs_card_data.GU_CD.isin([140,350])]
data_daegu = rs_card_data[rs_card_data.GU_CD.isin([260,110])]

In [25]:
seoul_table = data_seoul.pivot_table(values=['USE_CNT', 'USE_AMT'],
                         columns='year',   index=['week', 'MCT_CAT_CD'],
                         aggfunc='sum')

daegu_table = data_daegu.pivot_table(values=['USE_CNT', 'USE_AMT'],
                         columns='year',   index=['week', 'MCT_CAT_CD'],
                         aggfunc='sum')

In [26]:
# 전체 이용금액 차이
seoul_total_diff = (seoul_table.iloc[:,1] - seoul_table.iloc[:,0])
daegu_total_diff = (daegu_table.iloc[:,1] - daegu_table.iloc[:,0])

# 건당 이용금액 차이
seoul_cnt_diff = (seoul_table.iloc[:,1] / seoul_table.iloc[:,3] - seoul_table.iloc[:,0] / seoul_table.iloc[:,2])
daegu_cnt_diff = (daegu_table.iloc[:,1] / daegu_table.iloc[:,3] - daegu_table.iloc[:,0] / daegu_table.iloc[:,2])

In [27]:
seoul_total_diff2 = seoul_total_diff.reset_index()
seoul_cnt_diff2 = seoul_cnt_diff.reset_index()

seoul_total_diff2.columns = ['week', 'MCT_CAT_CD', 'diff']
seoul_cnt_diff2.columns = ['week', 'MCT_CAT_CD', 'diff']


daegu_total_diff2 = daegu_total_diff.reset_index()
daegu_cnt_diff2 = daegu_cnt_diff.reset_index()

daegu_total_diff2.columns = ['week', 'MCT_CAT_CD', 'diff']
daegu_cnt_diff2.columns = ['week', 'MCT_CAT_CD', 'diff']

In [28]:
fig = px.line(seoul_total_diff2, x="week", y="diff", color='MCT_CAT_CD')

fig.update_layout(xaxis=dict(tickmode='array', tickvals=np.arange(0, len(test2)), ticktext=test2))
fig.show()

In [29]:
fig = px.line(daegu_total_diff2, x="week", y="diff", color='MCT_CAT_CD')

fig.update_layout(xaxis=dict(tickmode='array', tickvals=np.arange(0, len(test2)), ticktext=test2))
fig.show()

In [30]:
fig = px.line(seoul_cnt_diff2, x="week", y="diff", color='MCT_CAT_CD')

fig.update_layout(xaxis=dict(tickmode='array', tickvals=np.arange(0, len(test2)), ticktext=test2))
fig.show()

In [31]:
fig = px.line(daegu_cnt_diff2, x="week", y="diff", color='MCT_CAT_CD')

fig.update_layout(xaxis=dict(tickmode='array', tickvals=np.arange(0, len(test2)), ticktext=test2))
fig.show()

## 생각해볼 거리

업종 코드를 간소화 하거나 일부만 봐야 할듯 

평균을 사용할지 합산을 사용할지 생각해 봐야 할듯

주당으로 하나로 볼지 주중 주말로 따로 볼지 생각해보기

- 일별은 의미 없어보임, 주별로 하면 주중 주말 효과를 한번에 해결할 장점 있음, 주중 주말을 따로 보면 더 깊게 세부적으로 본다는 장점이 있음


# 추가

## 변동폭 동일한 그래프

In [32]:
table

USE_AMT          USE_CNT       
year                2019     2020    2019   2020
week MCT_CAT_CD                                 
5    가구            94818   206005     336    272
     가전           820559   531576    3162   1188
     광학제품         129704   132828    2253   1194
     레저업소        1384626   940303   84124  59132
     레저용품        3116078  1675796   39799  21592
...                  ...      ...     ...    ...
22   자동차정비       1971042  2776705   63617  78834
     자동차판매        538584   661104    1035   1636
     전기            40665    64844     198    355
     주방용구         466752   872428    2484   6111
     직물           416783   809153    2601   6457

[414 rows x 4 columns]

In [33]:
var = (rs_card_data.groupby('MCT_CAT_CD').USE_AMT.sum() / rs_card_data.groupby('MCT_CAT_CD').USE_CNT.sum()).reset_index()

In [34]:
var.columns = ['MCT_CAT_CD','var']

In [35]:
std_cnt_diff2 = cnt_diff2.merge(var)

In [36]:
std_cnt_diff2['std_diff'] = std_cnt_diff2['diff'] / std_cnt_diff2['var']

In [37]:
fig = px.line(std_cnt_diff2, x="week", y="std_diff", color='MCT_CAT_CD')

fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = np.arange(0, len(test2)),
        ticktext = test2
    )
)
fig.show()

In [46]:
var2 = (rs_card_data.groupby('MCT_CAT_CD').USE_AMT.sum()).reset_index()

In [47]:
var2.columns = ['MCT_CAT_CD','var']

In [48]:
std_total_diff2 = total_diff2.merge(var2)

In [49]:
std_total_diff2['std_diff'] = std_total_diff2['diff'] / std_total_diff2['var']

In [50]:
fig = px.line(std_total_diff2, x="week", y="std_diff", color='MCT_CAT_CD')

fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = np.arange(0, len(test2)),
        ticktext = test2
    )
)
fig.show()